# requests

> This module provides a wrapper for interating with SQLite Database(Chosen datastore for qgofer)

In [ ]:
# | default_exp requests

In [ ]:
# | hide
import nbdev
from nbdev.showdoc import *

In [ ]:

# | export
import requests
import orjson
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

try:
    from qgoferutils.logger import get_logger
except:
    pass

In [ ]:
# | hide
%run 00_core.ipynb

In [ ]:
# | export


def get_failed_response(
    error_message="Some kind of API error ccured while interacting with the given URL",
) -> requests.Response:
    failed_response = requests.Response()
    failed_response.status_code = 500
    failed_response.reason = error_message
    failed_response._content = orjson.dumps({"message": f"{error_message}"})
    return failed_response


async def make_api_request(
    url: str,
    http_method: str = "GET",
    headers: dict = {},
    data: dict = {},
    auth: tuple = (),
    cookies: dict = {},
    params: dict = {},
) -> requests.Response:
    """Makes an API request to the given url with the given parameters."""
    if not all(headers.values()):
        return get_failed_response()
    s = requests.Session()
    retries = Retry(
        total=10,
        backoff_factor=0.1,
        status_forcelist=[403, 406, 408, 413, 429, 500, 502, 503, 504],
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://", HTTPAdapter(max_retries=retries))

    try:
        req = requests.Request(
            http_method,
            url,
            data=data,
            headers=headers,
            auth=auth,
            cookies=cookies,
            params=params,
        )
        prepped = req.prepare()
        resp = s.send(prepped)
        return resp
    except Exception as e:
        get_logger().error(f"Connection error while fetching data {e}")
        return get_failed_response()

In [ ]:
#| hide
nbdev.nbdev_export()